In [ ]:
!pip install gym stable_baselines3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 16.2 MB/s eta 0:00:00


In [ ]:
!pip install stable-baselines3[extra] gym


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=61ea886a6ab05a3e22ace0ce2eddecccbc7f6d8d55a662cac0215444018f1953
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
!pip install 'shimmy>=0.2.1'


In [ ]:
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO

class JobSchedulingEnv(gym.Env):
    def __init__(self):
        super(JobSchedulingEnv, self).__init__()
        self.num_jobs = 6
        self.job_durations = [2, 3, 5, 6, 2, 3]
        self.action_space = spaces.Discrete(self.num_jobs)
        self.observation_space = spaces.MultiBinary(self.num_jobs)
        self.max_steps = 100  # Maximum number of steps per episode
        self.reset()

    def step(self, action):
        # Toggle the machine assignment for the selected job
        self.state[action] = 1 - self.state[action]

        # Calculate the makespan for each machine
        makespan_m1 = sum([duration for i, duration in enumerate(self.job_durations) if self.state[i] == 0])
        makespan_m2 = sum([duration for i, duration in enumerate(self.job_durations) if self.state[i] == 1])
        new_makespan = max(makespan_m1, makespan_m2)

        # Calculate reward based on the change in makespan
        reward = self.current_makespan - new_makespan
        self.current_makespan = new_makespan

        # Increment the step count and check for termination
        self.current_step += 1
        done = self.current_step >= self.max_steps

        return self.state, reward, done, {}

    def reset(self):
        self.state = np.zeros(self.num_jobs, dtype=int)
        self.current_makespan = sum(self.job_durations)
        self.current_step = 0
        return self.state

    def render(self, mode='human', close=False):
        if close:
            return

        # Assign jobs to each machine based on the current state
        jobs_on_m1 = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == 0]
        jobs_on_m2 = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == 1]

        # Calculate makespan for each machine
        makespan_m1 = sum([duration for i, duration in enumerate(self.job_durations) if self.state[i] == 0])
        makespan_m2 = sum([duration for i, duration in enumerate(self.job_durations) if self.state[i] == 1])

        # Print the scheduling status
        print(f"Machine 1 (M1) - Jobs: {', '.join(jobs_on_m1)} | Makespan: {makespan_m1} minutes")
        print(f"Machine 2 (M2) - Jobs: {', '.join(jobs_on_m2)} | Makespan: {makespan_m2} minutes")
        print("-" * 50)

    # Method to set a specific initial state (optional)
    def set_initial_state(self, initial_state):
        if len(initial_state) == self.num_jobs:
            self.state = np.array(initial_state, dtype=int)

# Initialize the environment and the model
env = JobSchedulingEnv()
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Optionally, set a specific problem before testing
# env.set_initial_state([0, 0, 1, 1, 0, 1]) # Example initial state

# Test the trained agent
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    if dones:
        obs = env.reset()
    env.render()

# Save the model
model.save("job_scheduling_model")


In [ ]:
from stable_baselines3 import PPO

# Load the trained model
model = PPO.load("job_scheduling_model")

# Update the job durations for the new problem
new_job_durations = [2.30, 4.12, 7, 6, 2, 3]  # Replace with your job durations

# Create a new environment with the updated job durations
class NewJobSchedulingEnv(JobSchedulingEnv):
    def __init__(self):
        super().__init__()
        self.job_durations = new_job_durations  # Update the job durations

# Initialize the new environment
new_env = NewJobSchedulingEnv()

# Initialize variables to track the optimal solution
optimal_makespan = float('inf')
optimal_state = None

# Test the trained agent on the new environment
obs = new_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = new_env.step(action)

    # Check if the current solution is better than the best found so far
    if new_env.current_makespan < optimal_makespan:
        optimal_makespan = new_env.current_makespan
        optimal_state = obs.copy()

    if dones:
        obs = new_env.reset()

# Print the optimal solution
jobs_on_m1 = [f"J{i+1}" for i in range(new_env.num_jobs) if optimal_state[i] == 0]
jobs_on_m2 = [f"J{i+1}" for i in range(new_env.num_jobs) if optimal_state[i] == 1]

print("Optimal Solution:")
print(f"Machine 1 (M1) - Jobs: {', '.join(jobs_on_m1)} | Makespan: {sum(new_env.job_durations[i] for i in range(new_env.num_jobs) if optimal_state[i] == 0)} minutes")
print(f"Machine 2 (M2) - Jobs: {', '.join(jobs_on_m2)} | Makespan: {sum(new_env.job_durations[i] for i in range(new_env.num_jobs) if optimal_state[i] == 1)} minutes")


In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO

class JobSchedulingEnv(gym.Env):
    def __init__(self, num_jobs=6, job_durations=[2, 3, 5, 6, 2, 3], num_machines=2):
        super(JobSchedulingEnv, self).__init__()
        self.num_jobs = num_jobs
        self.job_durations = job_durations
        self.num_machines = num_machines

        # Action space: Each element in the action array represents a job's assigned machine
        self.action_space = spaces.MultiDiscrete([num_machines] * num_jobs)

        # Observation space: Each job's current machine assignment
        self.observation_space = spaces.MultiDiscrete([num_machines] * num_jobs)

        self.max_steps = 2000
        self.reset()

    def step(self, action):
        # Save the previous maximum makespan before updating the state
        prev_max_makespan = max([sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)])

        # Update the state based on the action
        self.state = action

        # Calculate the new makespan for each machine
        makespans = [sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)]
        new_max_makespan = max(makespans)

        # Calculate reward based on the change in the maximum makespan
        reward = prev_max_makespan - new_max_makespan

        # Increment the step count and check if the episode is done
        self.current_step += 1
        done = self.current_step >= self.max_steps

        return self.state, reward, done, {}


    def reset(self):
        self.state = np.zeros(self.num_jobs, dtype=int)
        self.current_makespan = sum(self.job_durations)
        self.current_step = 0
        return self.state

    def render(self, mode='human', close=False):
        if close:
            return

        for m in range(self.num_machines):
            jobs_on_machine = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == m]
            makespan = sum(self.job_durations[i] for i in range(self.num_jobs) if self.state[i] == m)
            print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")
        print("-" * 50)



number_of_epochs = 10  # Define the number of epochs
timesteps_per_epoch = 2000  # Define the number of timesteps per epoch
num_jobs = 6
num_machines = 3

# Initialize the environment with initial job durations
initial_job_durations = [random.uniform(1, 12) for _ in range(num_jobs)]
env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=initial_job_durations, num_machines=num_machines)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize the model
model = PPO("MlpPolicy", env, learning_rate=0.00025, n_steps=2048, batch_size=64,
            gamma=0.99, gae_lambda=0.95, clip_range=0.2, ent_coef=0.01,
            verbose=1, tensorboard_log="./ppo_job_scheduling_tensorboard/")

# Train the model over multiple epochs with different job durations
for epoch in range(number_of_epochs):
    # Generate new job durations for this epoch
    new_job_durations = [random.uniform(1, 12) for _ in range(num_jobs)]

    # Update the environment with new job durations
    env.envs[0].env.job_durations = new_job_durations

    # Continue training the model
    model.learn(total_timesteps=timesteps_per_epoch)

    # Optional: Save the model after each epoch
model_filename = f"job_scheduling_model_epoch_{num_machines}machines_{num_jobs}jobs"
model.save(model_filename)


In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


class JobSchedulingEnv(gym.Env):
    def __init__(self, num_jobs=6, job_durations=[2, 3, 5, 6, 2, 3], num_machines=2):
        super(JobSchedulingEnv, self).__init__()
        self.num_jobs = num_jobs
        self.job_durations = job_durations
        self.num_machines = num_machines
        self.action_space = spaces.MultiDiscrete([num_machines] * num_jobs)
        self.observation_space = spaces.MultiDiscrete([num_machines] * num_jobs)
        self.max_steps = 2000
        self.reset()

    def step(self, action):
        prev_max_makespan = max([sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)])
        self.state = action
        makespans = [sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)]
        new_max_makespan = max(makespans)
        reward = prev_max_makespan - new_max_makespan
        self.current_step += 1
        done = self.current_step >= self.max_steps
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.zeros(self.num_jobs, dtype=int)
        self.current_makespan = sum(self.job_durations)
        self.current_step = 0
        return self.state

    def render(self, mode='human', close=False):
        if close:
            return
        for m in range(self.num_machines):
            jobs_on_machine = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == m]
            makespan = sum(self.job_durations[i] for i in range(self.num_jobs) if self.state[i] == m)
            print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")
        print("-" * 50)

number_of_epochs = 50
timesteps_per_epoch = 2000
num_jobs = 5
num_machines = 3



training_scenarios = [
    [random.uniform(1, 12) for _ in range(num_jobs)] for _ in range(number_of_epochs)
]

initial_job_durations = training_scenarios[0]
env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=initial_job_durations, num_machines=num_machines)
env = make_vec_env(lambda: env, n_envs=1)

model = PPO("MlpPolicy", env, learning_rate=0.00025, n_steps=2048, batch_size=64,
            gamma=0.99, gae_lambda=0.95, clip_range=0.2, ent_coef=0.01,
            verbose=1, tensorboard_log="./ppo_job_scheduling_tensorboard/")
# Evaluation callback for logging performance and progress
eval_env = make_vec_env(lambda: JobSchedulingEnv(num_jobs=num_jobs, job_durations=initial_job_durations, num_machines=num_machines), n_envs=1)
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=True, render=False)

# Checkpoint callback for saving the model
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./logs/',
                                         name_prefix='rl_model')


for epoch, new_job_durations in enumerate(training_scenarios):
    env.envs[0].env.job_durations = new_job_durations
    model.learn(total_timesteps=timesteps_per_epoch)

model_filename = f"job_scheduling_model_epoch_{num_machines}machines_{num_jobs}jobs"
model.save(model_filename)


Using cuda device
Logging to ./ppo_job_scheduling_tensorboard/PPO_2


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2e+03    |
|    ep_rew_mean     | 20.6     |
| time/              |          |
|    fps             | 230      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
Logging to ./ppo_job_scheduling_tensorboard/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2e+03    |
|    ep_rew_mean     | 20.6     |
| time/              |          |
|    fps             | 230      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
Logging to ./ppo_job_scheduling_tensorboard/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2e+03    |
|    ep_rew_mean     | 21.5     |
| time/              |          |
|    fps             | 228      |
|    iterations 

In [ ]:
# Load the model- working testing model
loaded_model = PPO.load("job_scheduling_model_epoch_3machines_5jobs")

# Create an instance of the environment for testing
num_jobs = 5
job_durations = [10, 3, 20, 8, 1]  # These should match the training setup
num_machines = 3
test_env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=job_durations, num_machines=num_machines)

# Initialize variables to track the optimal solution
optimal_makespan = float('inf')
optimal_state = None

# Run the model to find the optimal solution
obs = test_env.reset()
for _ in range(20000):
    # Introduce a small probability of random action to allow exploration
    if random.random() < 0.05:  # 5% chance of random action
        action = test_env.action_space.sample()
    else:
        action, _states = loaded_model.predict(obs, deterministic=False)

    obs, _, dones, _ = test_env.step(action)

    # Track the best solution
    if test_env.current_makespan < optimal_makespan:
        optimal_makespan = test_env.current_makespan
        optimal_state = obs.copy()

    if dones:
        obs = test_env.reset()

# Print the optimal solution
print("Optimal Schedule:")
for m in range(num_machines):
    jobs_on_machine = [f"J{i+1}" for i in range(num_jobs) if optimal_state[i] == m]
    makespan = sum(test_env.job_durations[i] for i in range(num_jobs) if optimal_state[i] == m)
    print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")


Optimal Schedule:
Machine 1 - Jobs: J2, J4, J5 | Makespan: 12 minutes
Machine 2 - Jobs: J1 | Makespan: 10 minutes
Machine 3 - Jobs: J3 | Makespan: 20 minutes


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./ppo_job_scheduling_tensorboard/


In [ ]:
# Load the model itioal development
loaded_model = PPO.load("job_scheduling_model", env=env)

# Create an instance of the original environment for rendering
render_env = JobSchedulingEnv(num_jobs=8, job_durations=[2, 1, 4, 3, 5, 2, 6, 3], num_machines=3)

# Test the loaded model
obs = env.reset()
for i in range(200000):
    action, _states = loaded_model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)

    # Synchronize the state of the rendering environment
    render_env.state = env.get_attr("state")[0]
    render_env.current_makespan = env.get_attr("current_makespan")[0]

    # Use the render method of the original environment
    render_env.render()

    if dones:
        obs = env.reset()


In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

class JobSchedulingEnv(gym.Env):
    def __init__(self, num_jobs=6, job_durations=[2, 3, 5, 6, 2, 3], num_machines=2):
        super(JobSchedulingEnv, self).__init__()
        self.num_jobs = num_jobs
        self.job_durations = job_durations
        self.num_machines = num_machines
        self.action_space = spaces.MultiDiscrete([num_machines] * num_jobs)
        self.observation_space = spaces.MultiDiscrete([num_machines] * num_jobs)
        self.max_steps = 2000
        self.reset()

    def step(self, action):
        prev_max_makespan = max([sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)])
        self.state = action
        makespans = [sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)]
        new_max_makespan = max(makespans)
        reward = prev_max_makespan - new_max_makespan
        self.current_step += 1
        done = self.current_step >= self.max_steps
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.zeros(self.num_jobs, dtype=int)
        self.current_makespan = sum(self.job_durations)
        self.current_step = 0
        return self.state

    def render(self, mode='human', close=False):
        if close:
            return
        for m in range(self.num_machines):
            jobs_on_machine = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == m]
            makespan = sum(self.job_durations[i] for i in range(self.num_jobs) if self.state[i] == m)
            print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")
        print("-" * 50)

# Initialize your environment parameters
number_of_epochs = 100
timesteps_per_epoch = 2000
num_jobs = 5
num_machines = 3

# Epsilon-Greedy Parameters
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995

def select_action(model, observation, epsilon, env):
    if random.random() < epsilon:
        # Generate a random action for each environment in the batch
        return [env.action_space.sample() for _ in range(env.num_envs)]
    else:
        # Predict action using the model for each environment in the batch
        return model.predict(observation, deterministic=True)[0]


# Generating training scenarios
training_scenarios = [
    [random.uniform(1, 12) for _ in range(num_jobs)] for _ in range(number_of_epochs)
]

initial_job_durations = training_scenarios[0]
env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=initial_job_durations, num_machines=num_machines)
env = make_vec_env(lambda: env, n_envs=1)

# Initialize the PPO model
model = PPO("MlpPolicy", env, learning_rate=0.0025, n_steps=2048, batch_size=64,
            gamma=0.99, gae_lambda=0.95, clip_range=0.2, ent_coef=0.01,
            verbose=1, tensorboard_log="./ppo_job_scheduling_tensorboard/")

# Training loop with epsilon-greedy exploration
epsilon = epsilon_start
for epoch in range(number_of_epochs):
    obs = env.reset()
    for step in range(timesteps_per_epoch):
        action = select_action(model, obs, epsilon, env)
        obs, rewards, dones, infos = env.step(action)
        # ... (additional code for your training step) ...

    # Decay epsilon
    epsilon = max(epsilon_end, epsilon_decay * epsilon)

# Saving the model
model_filename = f"job_scheduling_model_epoch_{num_machines}machines_{num_jobs}jobs"
model.save(model_filename)

# ... [Your existing code for loading the model and testing] ...



Using cuda device


In [ ]:
# Load the model- working testing model
loaded_model = PPO.load("job_scheduling_model_epoch_3machines_5jobs")

# Create an instance of the environment for testing
num_jobs = 5
job_durations = [10, 3, 20, 8, 1]  # These should match the training setup
num_machines = 3
test_env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=job_durations, num_machines=num_machines)

# Initialize variables to track the optimal solution
optimal_makespan = float('inf')
optimal_state = None

# Run the model to find the optimal solution
obs = test_env.reset()
for _ in range(1000):
    # Introduce a small probability of random action to allow exploration
    if random.random() < 0.05:  # 5% chance of random action
        action = test_env.action_space.sample()
    else:
        action, _states = loaded_model.predict(obs, deterministic=False)

    obs, _, dones, _ = test_env.step(action)

    # Track the best solution
    if test_env.current_makespan < optimal_makespan:
        optimal_makespan = test_env.current_makespan
        optimal_state = obs.copy()

    if dones:
        obs = test_env.reset()

# Print the optimal solution
print("Optimal Schedule:")
for m in range(num_machines):
    jobs_on_machine = [f"J{i+1}" for i in range(num_jobs) if optimal_state[i] == m]
    makespan = sum(test_env.job_durations[i] for i in range(num_jobs) if optimal_state[i] == m)
    print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")


NameError: name 'PPO' is not defined

In [ ]:
!pip install pulp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 38.8 MB/s eta 0:00:00


In [ ]:
import random

# Parameters
num_jobs = 5
job_durations = [10, 3, 20, 8, 1]
num_machines = 3
population_size = 50
generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

# Initialize population
def initialize_population(population_size, num_jobs, num_machines):
    return [[random.randint(0, num_machines - 1) for _ in range(num_jobs)] for _ in range(population_size)]

# Calculate makespan
def calculate_makespan(chromosome, job_durations, num_machines):
    machine_times = [0] * num_machines
    for job, machine in enumerate(chromosome):
        machine_times[machine] += job_durations[job]
    return max(machine_times)

# Selection - Tournament selection
def tournament_selection(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = [random.choice(range(len(population))) for _ in range(tournament_size)]
        fittest_individual = min(tournament, key=lambda i: fitness[i])
        selected.append(population[fittest_individual])
    return selected

# Crossover - Single point crossover
def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    else:
        return parent1, parent2

# Mutation - Randomly change a job's machine assignment
def mutate(chromosome, num_machines, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.randint(0, num_machines - 1)
    return chromosome

# Main Genetic Algorithm
population = initialize_population(population_size, num_jobs, num_machines)

for generation in range(generations):
    # Calculate fitness for each individual
    fitness = [calculate_makespan(individual, job_durations, num_machines) for individual in population]

    # Selection
    selected = tournament_selection(population, fitness)

    # Crossover
    offspring = []
    for i in range(0, len(selected), 2):
        parent1, parent2 = selected[i], selected[i + 1]
        child1, child2 = crossover(parent1, parent2)
        offspring.extend([child1, child2])

    # Mutation
    population = [mutate(individual, num_machines, mutation_rate) for individual in offspring]

# Find the best solution
best_solution = min(population, key=lambda chrom: calculate_makespan(chrom, job_durations, num_machines))
best_makespan = calculate_makespan(best_solution, job_durations, num_machines)

print("Best Schedule:", best_solution)
print("Best Makespan:", best_makespan)


Best Schedule: [1, 2, 0, 2, 1]
Best Makespan: 20


In [ ]:
import gym
from gym import spaces
import numpy as np
import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import random

# Genetic Algorithm Functions
def initialize_population(population_size, num_jobs, num_machines):
    return [[random.randint(0, num_machines - 1) for _ in range(num_jobs)] for _ in range(population_size)]

def calculate_makespan(chromosome, job_durations, num_machines):
    machine_times = [0] * num_machines
    for job, machine in enumerate(chromosome):
        machine_times[machine] += job_durations[job]
    return max(machine_times)

def tournament_selection(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = [random.choice(range(len(population))) for _ in range(tournament_size)]
        fittest_individual = min(tournament, key=lambda i: fitness[i])
        selected.append(population[fittest_individual])
    return selected

def crossover(parent1, parent2, crossover_rate):
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    else:
        return parent1, parent2

def mutate(chromosome, num_machines, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.randint(0, num_machines - 1)
    return chromosome

def run_genetic_algorithm(num_jobs, job_durations, num_machines, population_size, generations, crossover_rate, mutation_rate):
    population = initialize_population(population_size, num_jobs, num_machines)

    for generation in range(generations):
        fitness = [calculate_makespan(individual, job_durations, num_machines) for individual in population]
        selected = tournament_selection(population, fitness)
        offspring = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            offspring.extend([child1, child2])
        population = [mutate(individual, num_machines, mutation_rate) for individual in offspring]

    best_solution = min(population, key=lambda chrom: calculate_makespan(chrom, job_durations, num_machines))
    best_makespan = calculate_makespan(best_solution, job_durations, num_machines)
    print(best_makespan)
    return best_makespan



class JobSchedulingEnv(gym.Env):
    def __init__(self, num_jobs=5, job_durations=[10, 3, 20, 8, 1], num_machines=3, target_makespan=20,tolerance=1):
        super(JobSchedulingEnv, self).__init__()
        self.num_jobs = num_jobs
        self.job_durations = job_durations
        self.num_machines = num_machines
        self.target_makespan = target_makespan
        self.action_space = spaces.Discrete(num_jobs * num_machines)  # New action space
        self.observation_space = spaces.Box(low=0, high=num_machines, shape=(num_jobs,), dtype=np.int32)
        self.max_steps = 2000
        self.tolerance = tolerance
        self.state = None
        self.reset()

    def reset(self):
        self.state = np.random.randint(low=0, high=self.num_machines, size=self.num_jobs)  # Random initial state
        self.current_step = 0
        return self.state

    def step(self, action):
        job_index = action // self.num_machines
        machine_index = action % self.num_machines
        self.state[job_index] = machine_index
        current_makespan = max([sum(self.job_durations[j] for j in range(self.num_jobs) if self.state[j] == m) for m in range(self.num_machines)])
        reward = -abs(current_makespan - self.target_makespan)  # Negative absolute difference as reward
        self.current_step += 1
        done = abs(current_makespan - self.target_makespan) <= self.tolerance
        return self.state, reward, done, {}


    def render(self, mode='human', close=False):
        if close:
            return
        for m in range(self.num_machines):
            jobs_on_machine = [f"J{i+1}" for i in range(self.num_jobs) if self.state[i] == m]
            makespan = sum(self.job_durations[i] for i in range(self.num_jobs) if self.state[i] == m)
            print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")
        print("-" * 50)

# Training Parameters
number_of_epochs = 3
timesteps_per_epoch = 40000
num_jobs = 5
num_machines = 3
population_size = 50
generations = 100
crossover_rate = 0.8
mutation_rate = 0.1
tolerance = 5

# Initialize the PPO model
dummy_env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=[1] * num_jobs, num_machines=num_machines, target_makespan=1,tolerance=tolerance)
model = PPO("MlpPolicy", dummy_env, learning_rate=0.00025, n_steps=2000, batch_size=64, gamma=0.99, gae_lambda=0.95, clip_range=0.2, ent_coef=0.01, verbose=1, tensorboard_log="./ppo_job_scheduling_tensorboard/")

# Training loop
for epoch in range(number_of_epochs):
    # Generate random job durations for each epoch
    job_durations = [random.randint(1, 5) for _ in range(num_jobs)]

    # Run the Genetic Algorithm to find the target makespan
    target_makespan = run_genetic_algorithm(num_jobs, job_durations, num_machines, population_size, generations, crossover_rate, mutation_rate)


    # Create the real environment with new parameters
    real_env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=job_durations, num_machines=num_machines, target_makespan=target_makespan,tolerance=tolerance)
    real_env = make_vec_env(lambda: real_env, n_envs=1)

    # Update the model's environment
    model.set_env(real_env)

    # Train the model
    model.learn(total_timesteps=timesteps_per_epoch)


# Saving the model
model.save("job_scheduling_model")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
3
Logging to ./ppo_job_scheduling_tensorboard/PPO_290


/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2000`, after every 31 untruncated mini-batches, there will be a truncated mini-batch of size 16
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2000 and n_envs=1)
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.04     |
|    ep_rew_mean     | -2.58    |
| time/              |          |
|    fps             | 684      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02        |
|    ep_rew_mean          | -2.41       |
| time/                   |             |
|    fps                  | 553         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.018658869 |
|    clip_fraction        | 0.149       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.7        |
|    explained_variance   | -0.0262     |
|    learning_rate        | 0.

In [ ]:
from stable_baselines3 import PPO

# Load the trained model
model = PPO.load("job_scheduling_model")

# Test job durations and environment setup
test_job_durations = [1, 2, 3, 5, 1]  # Example job durations
num_jobs = len(test_job_durations)
num_machines = 3
test_env = JobSchedulingEnv(num_jobs=num_jobs, job_durations=test_job_durations, num_machines=num_machines, target_makespan=20, tolerance=1)

# Run the model in the test environment
obs = test_env.reset()
done = False
max_iterations = 30000  # Prevent infinite loop
iteration = 0

while not done and iteration < max_iterations:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, info = test_env.step(action)
    iteration += 1


# Check if loop exited due to reaching max iterations
if iteration >= max_iterations:
    print("Reached maximum iterations without fulfilling termination conditions.")

# Print the schedule
def print_schedule(env):
    print("\nOptimal Schedule:")
    for m in range(env.num_machines):
        jobs_on_machine = [f"J{i+1}" for i in range(env.num_jobs) if env.state[i] == m]
        makespan = sum(env.job_durations[i] for i in range(env.num_jobs) if env.state[i] == m)
        print(f"Machine {m+1} - Jobs: {', '.join(jobs_on_machine)} | Makespan: {makespan} minutes")

print_schedule(test_env)


Reached maximum iterations without fulfilling termination conditions.

Optimal Schedule:
Machine 1 - Jobs: J5 | Makespan: 1 minutes
Machine 2 - Jobs: J1, J3 | Makespan: 4 minutes
Machine 3 - Jobs: J2, J4 | Makespan: 7 minutes


In [83]:
import random

# Parameters
num_jobs = 10
job_durations = [10, 3, 20, 8, 1,10, 3, 20, 8, 1]
num_machines = 5
population_size = 50
generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

# Initialize population
def initialize_population(population_size, num_jobs, num_machines):
    return [[random.randint(0, num_machines - 1) for _ in range(num_jobs)] for _ in range(population_size)]

# Calculate makespan and balance
def calculate_makespan_and_balance(chromosome, job_durations, num_machines):
    machine_times = [0] * num_machines
    for job, machine in enumerate(chromosome):
        machine_times[machine] += job_durations[job]
    max_makespan = max(machine_times)
    balance_penalty = sum([(max_makespan - time)**2 for time in machine_times])  # Penalize unbalanced schedules
    return max_makespan + balance_penalty

# Tournament selection
def tournament_selection(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = [random.choice(range(len(population))) for _ in range(tournament_size)]
        fittest_individual = min(tournament, key=lambda i: fitness[i])
        selected.append(population[fittest_individual])
    return selected

# Crossover - Single point crossover
def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    else:
        return parent1, parent2

# Mutation - Randomly change a job's machine assignment
def mutate(chromosome, num_machines, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.randint(0, num_machines - 1)
    return chromosome

# Function to create a readable schedule from the chromosome
def create_schedule(chromosome, job_durations):
    schedule = {machine: [] for machine in range(num_machines)}
    for job, machine in enumerate(chromosome):
        schedule[machine].append((f"J{job+1}", job_durations[job]))
    return schedule

# Main Genetic Algorithm
population = initialize_population(population_size, num_jobs, num_machines)

for generation in range(generations):
    fitness = [calculate_makespan_and_balance(individual, job_durations, num_machines) for individual in population]
    selected = tournament_selection(population, fitness)
    offspring = []
    for i in range(0, len(selected), 2):
        parent1, parent2 = selected[i], selected[i + 1]
        child1, child2 = crossover(parent1, parent2)
        offspring.extend([child1, child2])
    population = [mutate(individual, num_machines, mutation_rate) for individual in offspring]

# Find the best solution and create schedule
best_solution = min(population, key=lambda chrom: calculate_makespan_and_balance(chrom, job_durations, num_machines))
best_schedule = create_schedule(best_solution, job_durations)
print(best_schedule)
# Displaying the schedule
print("Optimal Schedule:")
for machine, jobs in best_schedule.items():
    job_list = ', '.join([job[0] for job in jobs])
    makespan = sum([job[1] for job in jobs])
    print(f"Machine {machine + 1} - Jobs: {job_list} | Makespan: {makespan} minutes")


{0: [('J4', 8), ('J9', 8)], 1: [('J1', 10), ('J5', 1), ('J7', 3)], 2: [('J2', 3), ('J6', 10), ('J10', 1)], 3: [('J8', 20)], 4: [('J3', 20)]}
Optimal Schedule:
Machine 1 - Jobs: J4, J9 | Makespan: 16 minutes
Machine 2 - Jobs: J1, J5, J7 | Makespan: 14 minutes
Machine 3 - Jobs: J2, J6, J10 | Makespan: 14 minutes
Machine 4 - Jobs: J8 | Makespan: 20 minutes
Machine 5 - Jobs: J3 | Makespan: 20 minutes


In [84]:
# Revised approach to include the robot cell information in the schedule

# Original schedule
original_schedule = best_schedule

# User input for robot cells
user_input = {
    "R 1": [1, 3],
    "R 2": [2,5],
    "R 3": [4]

}

# Function to find the robot cell for a given machine
def find_robot_cell(machine_number, user_input):
    for cell_name, machines in user_input.items():
        if machine_number in machines:
            return cell_name
    return None

# Adding robot cell information to each schedule
for machine, jobs in original_schedule.items():
    robot_cell = find_robot_cell(machine + 1, user_input)  # +1 because machine numbering starts from 1
    original_schedule[machine] = (robot_cell, jobs)

# Re-arranging the schedule by robot cell
rearranged_schedule = dict(sorted(original_schedule.items(), key=lambda item: item[1][0]))

# Displaying the rearranged schedule
for machine, (cell, jobs) in rearranged_schedule.items():
    job_list = ', '.join([job[0] for job in jobs])
    makespan = sum([job[1] for job in jobs])
    print(f"Machine {machine} (in {cell}) - Jobs: {job_list} | Makespan: {makespan} minutes")

# Return rearranged_schedule for further analysis if needed
rearranged_schedule




Machine 0 (in R 1) - Jobs: J4, J9 | Makespan: 16 minutes
Machine 2 (in R 1) - Jobs: J2, J6, J10 | Makespan: 14 minutes
Machine 1 (in R 2) - Jobs: J1, J5, J7 | Makespan: 14 minutes
Machine 4 (in R 2) - Jobs: J3 | Makespan: 20 minutes
Machine 3 (in R 3) - Jobs: J8 | Makespan: 20 minutes


{0: ('R 1', [('J4', 8), ('J9', 8)]),
 2: ('R 1', [('J2', 3), ('J6', 10), ('J10', 1)]),
 1: ('R 2', [('J1', 10), ('J5', 1), ('J7', 3)]),
 4: ('R 2', [('J3', 20)]),
 3: ('R 3', [('J8', 20)])}

In [ ]:
# Original schedule with jobs
original_schedule = rearranged_schedule


# User input for subtasks of each job
job_subtasks = {
    'J1': [('T1', 4), ('T2', 3), ('T3', 3)],
    'J2': [('T1', 3)],
    'J3': [('T1', 7), ('T2', 8), ('T3', 5)],
    'J4': [('T1', 4), ('T2', 4)],
    'J5': [('T1', 1)],
    'J6': [('T1', 5), ('T2', 5)],
    'J7': [('T1', 1), ('T2', 1), ('T3', 1)],
    'J8': [('T1', 10), ('T2', 10)],
    'J9': [('T1', 4), ('T2', 4)],
    'J10': [('T1', 1)]
}


# Replace jobs with their corresponding subtasks in the schedule
for machine, (cell, jobs) in original_schedule.items():
    new_jobs = []
    for job, _ in jobs:
        if job in job_subtasks:
            for subtask in job_subtasks[job]:
                new_subtask = (f"{subtask[0]}{job[1:]}", subtask[1])  # Format: T12 for Task 2 of Job 1
                new_jobs.append(new_subtask)
    original_schedule[machine] = (cell, new_jobs)

# Displaying the updated schedule
for machine, (cell, jobs) in original_schedule.items():
    job_list = ', '.join([f"{job[0]} ({job[1]})" for job in jobs])
    print(f"Machine {machine} (in {cell}) - Tasks: {job_list}")

# Return original_schedule for further analysis if needed
original_schedule



In [73]:
# Re-defining the original schedule and job subtasks due to code execution state reset


# User input for subtasks of each job
job_subtasks = {
    'J1': [('T1', 4), ('T2', 3), ('T3', 3)],
    'J2': [('T1', 3)],
    'J3': [('T1', 7), ('T2', 8), ('T3', 5)],
    'J4': [('T1', 4), ('T2', 4)],
    'J5': [('T1', 1)],
    'J6': [('T1', 5), ('T2', 5)],
    'J7': [('T1', 1), ('T2', 1), ('T3', 1)],
    'J8': [('T1', 10), ('T2', 10)],
    'J9': [('T1', 4), ('T2', 4)],
    'J10': [('T1', 1)]
}

# Replace jobs with their corresponding subtasks in the schedule
for machine, (cell, jobs) in original_schedule.items():
    new_jobs = []
    for job, _ in jobs:
        if job in job_subtasks:
            for subtask in job_subtasks[job]:
                subtask_label = f"T{job[1:] if len(job) > 2 else job[1]}{subtask[0][1]}"  # Correct format: TXY
                new_jobs.append((subtask_label, subtask[1]))
    original_schedule[machine] = (cell, new_jobs)

# Displaying the updated schedule
for machine, (cell, jobs) in original_schedule.items():
    job_list = ', '.join([f"{job[0]} ({job[1]})" for job in jobs])
    print(f"Machine {machine} (in {cell}) - Tasks: {job_list}")

# Return original_schedule for further analysis if needed
original_schedule


Machine 0 (in R 1) - Tasks: T81 (10), T82 (10)
Machine 1 (in R 2) - Tasks: T51 (1), T61 (5), T62 (5), T71 (1), T72 (1), T73 (1)
Machine 2 (in R 1) - Tasks: T11 (4), T12 (3), T13 (3), T21 (3), T101 (1)
Machine 3 (in R 3) - Tasks: T31 (7), T32 (8), T33 (5)
Machine 4 (in R 2) - Tasks: T41 (4), T42 (4), T91 (4), T92 (4)


{0: ('R 1', [('T81', 10), ('T82', 10)]),
 1: ('R 2',
  [('T51', 1), ('T61', 5), ('T62', 5), ('T71', 1), ('T72', 1), ('T73', 1)]),
 2: ('R 1', [('T11', 4), ('T12', 3), ('T13', 3), ('T21', 3), ('T101', 1)]),
 3: ('R 3', [('T31', 7), ('T32', 8), ('T33', 5)]),
 4: ('R 2', [('T41', 4), ('T42', 4), ('T91', 4), ('T92', 4)])}

In [85]:
# Re-defining the original schedule and job subtasks due to code execution state reset

# User input for subtasks of each job
job_subtasks = {
    'J1': [('T1', 4,'S1'), ('T2', 3,'S3'), ('T3', 3),'S4'],
    'J2': [('T1', 3,'S1')],
    'J3': [('T1', 7,'S3'), ('T2', 8,'S4'), ('T3', 5,'S6')],
    'J4': [('T1', 4,'S1'), ('T2', 4,'S3')],
    'J5': [('T1', 1,'S1')],
    'J6': [('T1', 5,'S2'), ('T2', 5,'S3')],
    'J7': [('T1', 1,'S2'), ('T2', 1,'S2'), ('T3', 1,'S1')],
    'J8': [('T1', 10,'S4'), ('T2', 10,'S2')],
    'J9': [('T1', 4,'S1'), ('T2', 4,'S4')],
    'J10': [('T1', 1,'S4')]
}

# Replace jobs with their corresponding subtasks and tools in the schedule
for machine, (cell, jobs) in original_schedule.items():
    new_jobs = []
    for job_info in jobs:
        job, duration = job_info[0], job_info[1]  # Unpack job ID and duration
        if job in job_subtasks:
            for subtask in job_subtasks[job]:
                # Extract task number and job number more safely
                task_number = subtask[0][1:]  # Assuming task format is 'T<number>'
                job_number = job[1:]  # Extracting job number from job ID
                subtask_label = f"T{job_number}{task_number}"  # Correct format: TXY

                tool = subtask[2] if len(subtask) > 2 else 'None'  # Handling missing tool info
                new_jobs.append((subtask_label, subtask[1], tool))
    original_schedule[machine] = (cell, new_jobs)

# Displaying the updated schedule with tools
for machine, (cell, jobs) in original_schedule.items():
    job_list = ', '.join([f"{job[0]} ({job[1]} units, Tool: {job[2]})" for job in jobs])
    print(f"Machine {machine} (in {cell}) - Tasks: {job_list}")

# Return original_schedule for further analysis if needed
original_schedule


Machine 0 (in R 1) - Tasks: T41 (4 units, Tool: S1), T42 (4 units, Tool: S3), T91 (4 units, Tool: S1), T92 (4 units, Tool: S4)
Machine 1 (in R 2) - Tasks: T11 (4 units, Tool: S1), T12 (3 units, Tool: S3), T13 (3 units, Tool: None), T1 (4 units, Tool: None), T51 (1 units, Tool: S1), T71 (1 units, Tool: S2), T72 (1 units, Tool: S2), T73 (1 units, Tool: S1)
Machine 2 (in R 1) - Tasks: T21 (3 units, Tool: S1), T61 (5 units, Tool: S2), T62 (5 units, Tool: S3), T101 (1 units, Tool: S4)
Machine 3 (in R 3) - Tasks: T81 (10 units, Tool: S4), T82 (10 units, Tool: S2)
Machine 4 (in R 2) - Tasks: T31 (7 units, Tool: S3), T32 (8 units, Tool: S4), T33 (5 units, Tool: S6)


{0: ('R 1',
  [('T41', 4, 'S1'), ('T42', 4, 'S3'), ('T91', 4, 'S1'), ('T92', 4, 'S4')]),
 1: ('R 2',
  [('T11', 4, 'S1'),
   ('T12', 3, 'S3'),
   ('T13', 3, 'None'),
   ('T1', '4', 'None'),
   ('T51', 1, 'S1'),
   ('T71', 1, 'S2'),
   ('T72', 1, 'S2'),
   ('T73', 1, 'S1')]),
 2: ('R 1',
  [('T21', 3, 'S1'), ('T61', 5, 'S2'), ('T62', 5, 'S3'), ('T101', 1, 'S4')]),
 3: ('R 3', [('T81', 10, 'S4'), ('T82', 10, 'S2')]),
 4: ('R 2', [('T31', 7, 'S3'), ('T32', 8, 'S4'), ('T33', 5, 'S6')])}

In [86]:
# Original schedule with robotic cells and tasks


# Filter function for a specific robotic cell
def filter_schedule_by_robot_cell(schedule, cell_name):
    return {machine: (cell, tasks) for machine, (cell, tasks) in schedule.items() if cell == cell_name}

# Example: Filtering for Robotic Cell R1
filtered_schedule_R1 = filter_schedule_by_robot_cell(original_schedule, 'R 1')

# Displaying the filtered schedule for R1
for machine, (cell, tasks) in filtered_schedule_R1.items():
    task_list = ', '.join([f"{task[0]} ({task[1]})" for task in tasks])
    print(f"Machine {machine} (in {cell}) - Tasks: {task_list}")

# Return filtered_schedule_R1 for further analysis if needed
filtered_schedule_R1



Machine 0 (in R 1) - Tasks: T41 (4), T42 (4), T91 (4), T92 (4)
Machine 2 (in R 1) - Tasks: T21 (3), T61 (5), T62 (5), T101 (1)


{0: ('R 1',
  [('T41', 4, 'S1'), ('T42', 4, 'S3'), ('T91', 4, 'S1'), ('T92', 4, 'S4')]),
 2: ('R 1',
  [('T21', 3, 'S1'), ('T61', 5, 'S2'), ('T62', 5, 'S3'), ('T101', 1, 'S4')])}

In [90]:
def calculate_makespan_and_tool_change(schedule):
    machines = {}
    makespan = 0
    tool_changeover_time = 0

    for _, (job, tasks) in schedule.items():
        current_tool = None
        current_machine = None
        machine_time = 0

        for task in tasks:
            task_id, processing_time, tool = task

            if current_tool is None:
                current_tool = tool
                current_machine = job + current_tool
                machines[current_machine] = 0

            if current_tool != tool:
                tool_changeover_time += 5
                current_tool = tool

            if current_machine != job + current_tool:
                machine_time = machines.get(current_machine, 0)
                current_machine = job + current_tool

            machine_time += processing_time
            machines[current_machine] = machine_time
            makespan = max(makespan, machine_time)

    return makespan, tool_changeover_time

schedule = filtered_schedule_R1

makespan, tool_changeover_time = calculate_makespan_and_tool_change(schedule)

print("Makespan:", makespan)
print("Tool Changeover Time:", tool_changeover_time)

# Remove 'R 1' from each schedule entry
new_schedule = {key: value[1] for key, value in schedule.items()}

print(new_schedule)


Makespan: 16
Tool Changeover Time: 30
{0: [('T41', 4, 'S1'), ('T42', 4, 'S3'), ('T91', 4, 'S1'), ('T92', 4, 'S4')], 2: [('T21', 3, 'S1'), ('T61', 5, 'S2'), ('T62', 5, 'S3'), ('T101', 1, 'S4')]}


In [ ]:
schedule = {
    0: ('R 1', [('T41', 4, 'S1'), ('T42', 4, 'S3'), ('T91', 4, 'S1'), ('T92', 4, 'S4')]),
    2: ('R 1', [('T21', 3, 'S1'), ('T61', 5, 'S2'), ('T62', 5, 'S3'), ('T101', 1, 'S4')])
}

# Remove 'R 1' from each schedule entry
new_schedule = {key: value[1] for key, value in schedule.items()}

print(new_schedule)
